In [ ]:
import numpy as np
from scipy.sparse.linalg import eigs, LinearOperator
from ncon import ncon
from tutorialFunctions import createMPS, mixedCanonical, minAcC
from time import time
import matplotlib.pyplot as plt

# Tangent-space methods for uniform matrix product states

https://arxiv.org/abs/1810.07006

## 3. Transfer matrices and fixed points

### 3.1 The vumps algorithm for MPOs

Matrix product states have been used extensively as variational ansatz for ground states of local hamiltonians, but in the last years it has been observed that they can also provide accurate approximations for fixed points of transfer matrices. In this chapter we investigate tangent-space methods for one-dimensional transfer matrices.

A one-dimensional transfer matrix in the form of matrix product operator (MPO) can be represented diagrammatically as

<img src="img/transferMPO.png" alt="transferMPO" width="500">

Whenever we contract an infinite two-dimensional tensor network, we want to find the fixed point of this operator, i.e. we want to solve the fixed-point equation

$$ T(O) |\Psi> \propto |\Psi> $$

such that we can collapse the vertical direction of the network.

We now make the ansatz that the fixed point (leading eigenvector) of this operator is an MPS. Suppose now we have indeed found an MPS representation $|\Psi(A)>$ of the fixed point of $T(O)$, then the eigenvalue is given by

$$ \Lambda = <\Psi(A)| T |\Psi(A) > $$

Bringing the MPS in mixed canonical form, we write

<img src="img/partFunc.png" alt="partition function" width="450">

Again, contracting this infinite network requires that we find $F_L$ and $F_R$, the fixed points of the left and right channel operators, such that we may now collapse the horizontal direction. These are found by considering the following diagrams:

<img src="img/FL.png" alt="Fl" width="240">
<img src="img/FR.png" alt="Fr" width="260">

Finally these are properly normalised if we set
<img src="img/FlFr.png" alt="FlFr" width="220">

We therefore first require routines for finding and normalising these fixed points.

In [ ]:
def leftFixedPointMPO(O, Al, tol):
    """
    Computes the left fixed point (250).

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        tol : float, optional
            tolerance for eigenvalue solver

        Returns
        -------
        lam : float
            Leading left eigenvalue.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.

    """
    
    # given as an example
    
    D = Al.shape[0]
    d = Al.shape[1]
    
    # construct handle for the action of the relevant operator and cast to linear operator
    transferLeftHandleMPO = lambda v: (ncon((v.reshape((D,d,D)), Al, np.conj(Al), O),([5, 3, 1], [1, 2, -3], [5, 4, -1], [3, 2, -2, 4]))).reshape(-1)
    transferLeftMPO = LinearOperator((D**2*d, D**2*d), matvec=transferLeftHandleMPO)
    # compute the largest magnitude eigenvalue and corresponding eigenvector
    lam, Fl = eigs(transferLeftMPO, k=1, which="LM", tol=tol)
    
    return lam, Fl.reshape((D,d,D))


def rightFixedPointMPO(O, Ar, tol):
    """
    Computes the right fixed point (250).

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        tol : float, optional
            tolerance for eigenvalue solver

        Returns
        -------
        lam : float
            Leading right eigenvalue.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.

    """
    
    return lam, Fr


def overlapFixedPointsMPO(Fl, Fr, C):
    """
    Performs the contraction that gives the overlap of the fixed points (251).

        Parameters
        ----------
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.

        Returns
        -------
        overlap : float
            Overlap of the fixed points.

    """
    
    # given
    
    overlap = ncon((Fl, Fr, C, np.conj(C)), ([1, 3, 2], [5, 3, 4], [2, 5], [1, 4]))
    
    return overlap


We now want to use the VUMPS algorithm for finding this fixed point MPS of the MPO $T(O)$.

To do so, we calculate the tangent-space projector in mixed gauge and again find two terms that can be characterised by the maps

<img src="img/OAc.png" alt="OAC" width="330">
<img src="img/OC.png" alt="OC" width="300">

Together with the consistency conditions, a fixed point is thus characterised by the set of equations:

$$ O_{A_C}(A_C) \propto A_C $$
$$ O_C(C) \propto C $$
$$ A_C = A_L C = CA_R $$

The vumps algorithm for MPO's can then be formulated in a similar way as in the Hamiltonian case:
    (i) we start from a given MPS $\{A^i_L , A^i_R , A^i_C , C^i \}$
    (ii) determine $F_L$ and $F_R$
    (iii) solve the two eigenvalue equations obtaining $\tilde{A}_C$ and $\tilde{C}$, and
    (iv) determine the $A^{i+1}_L$ and $A^{i+1}_R$ that minimize $||\tilde{A} - A^{i+1}_L \tilde{C}||$ and $||\tilde{A} - \tilde{C}A^{i+1}_R||$.

We can already determin the left and right fixed points, so we now need the action of the maps $O_{A_C}$ and $O_C$ on a given tensor:

In [ ]:
def O_Ac(X, O, Fl, Fr, lam):
    """
    Action of the map (256) on a given tensor.

        Parameters
        ----------
        X : np.array(D, d, D)
            Tensor of size (D, d, D)
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        lam : float
            Leading eigenvalue.

        Returns
        -------
        Xnew : np.array(D, d, D)
            Result of the action of O_Ac on the tensor X.

    """
    
    # given as an example
    
    Xnew = ncon((Fl, Fr, X, O),([-1, 2, 1], [4, 5, -3], [1, 3, 4], [2, 3, 5, -2])) / lam
    
    return Xnew


def O_C(X, Fl, Fr):
    """
    Action of the map (257) on a given tensor.

        Parameters
        ----------
        X : np.array(D, D)
            Tensor of size (D, D)
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.

        Returns
        -------
        Xnew : np.array(D, d, D)
            Result of the action of O_C on the tensor X.

    """
    
    return Xnew

Solving the eigenvalue equations for these operators gives an update for the center tensors, $\tilde{A}_C$ and $\tilde{C}$:

In [ ]:
def calcNewCenterMPO(O, Ac, C, Fl, Fr, lam, tol=1e-3):
    """
    Find new guess for Ac and C as fixed points of the maps O_Ac and O_C.
    
        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Ac : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        lam : float
            Leading eigenvalue.
        tol : float, optional
            current tolerance
    
        Returns
        -------
        AcTilde : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        CTilde : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.

    """
    
    return AcTilde, CTilde

And after retrieving $\tilde{A}_C$ and $\tilde{C}$ we can use the same 'minAcC()' function as in the case of Hamiltonian vumps to iterate the whole procedure until convergence. These building blocks now allow to implement the vumps algorithm for MPOs (algorithm 8):

In [ ]:
def vumpsMPO(O, D, A0=None, tol=1e-4):
    """
    Find the fixed point MPS of a given MPO using VUMPS.
    
        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        D : int
            Bond dimension
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Relative convergence criterium.
        
        Returns
        -------
        lam : float
            Leading eigenvalue.
        Al : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        
    """
    
    return lam, Al, Ac, Ar, C, Fl, Fr

The partition function is eventually given by the left and right fixed point eigenvalue lambda. To check this we define some functions calculating the free energy and the magnetization for the simulation and the exact solution.

### 3.2 The 2d classical Ising model

We can use this algorithm to compute the partition function of the 2d classical Ising model, which can be written as a two-dimensional tensor network contraction. This is done by constructing a one-dimensional transfer matrix in the form of an MPO, and noting that the partition function is then an (infinite) product of these transfer matrices, stacked vertically.

We start by defining the tensors that make up the transfer matrix for the Ising model. If we consider an infinite square lattice, we may simulate the model by placing a spin $1/2$ degree of freedom on each vertex, and then characterising the interactions by inserting a matrix $Q$ with Boltzmann weights on each of the edges. In the graphical language of tensor networks this corresponds to having a 4-legged 'delta tensor' on the vertices and a 2-legged $Q$ on the bonds.

If we want to rephrase this in terms of the transfer matrix we defined above, we split every bond interaction matrix over the two adjacent vertices and absorp them in the definition of the MPO tensor $O$. The partition function is given by contracting a tensor network consisting of all these rank 4 MPO tensors $O$, which have dimensions $(d,d,d,d)$. We can find the MPS fixed point of the corresponding transfer matrix using the vumps algorithm, which allows to perform this infinite contraction efficiently.

Below we define this Ising MPO operator $O$, along with a magnetization operator $M$ that can be obtained by contracting the spin-1/2 $Z$ operator into a given MPO tensor. We then construct routines that compute the Ising partition function and magnetization (as the expectation value of the $M$ operator) for a given fixed point MPS. This can be compared with the exact Onsager solution. Doing this for a range of different temperatures then allows us to compute the magnetization and free energy in the Ising model as a functions of temperature:

In [ ]:
def isingO(beta, J):
    """
    Gives the MPO tensor corresponding to the partition function of the 2d 
    classical Ising model at a given temperature and coupling, obtained by
    distributing the Boltzmann weights evenly over all vertices.
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        O : np.array (2, 2, 2, 2)
            MPO tensor,
            ordered left-top-right-bottom.

    """
    # basic vertex tensor
    vertex = np.zeros( (2,) * 4 )
    vertex[tuple([np.arange(2)] * 4)] = 1
    # build square root of matrix of Boltzmann weights and pull into vertex edges
    c, s = np.sqrt(np.cosh(beta*J)), np.sqrt(np.sinh(beta*J))
    Qsqrt = 1/np.sqrt(2) * np.array([[c+s, c-s],[c-s, c+s]])
    O = ncon((Qsqrt, Qsqrt, Qsqrt, Qsqrt, vertex), ([-1,1], [-2,2], [-3,3], [-4,4], [1,2,3,4]))
    return O


def isingM(beta, J):
    """
    Gives the magnetizatopn MPO tensor for the 2d classical Ising model at a
    given temperature and coupling.
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        M : np.array (2, 2, 2, 2)
            Magnetization MPO tensor,
            ordered left-top-right-bottom.

    """
    vertex = np.zeros( (2,) * 4 )
    vertex[tuple([np.arange(2)] * 4)] = 1
    Z = np.array([[1,0],[0,-1]])
    c, s = np.sqrt(np.cosh(beta*J)), np.sqrt(np.sinh(beta*J))
    Qsqrt = 1/np.sqrt(2) * np.array([[c+s, c-s],[c-s, c+s]])
    vertexZ = ncon((Z, vertex), ([-1,1], [1,-2,-3,-4]))
    M = ncon((Qsqrt, Qsqrt, Qsqrt, Qsqrt, vertexZ), ([-1,1], [-2,2], [-3,3], [-4,4], [1,2,3,4]))
    return M


def isingMagnetization(beta, J, Ac, Fl, Fr):
    """
    Computes the expectation value of the magnetization in the Ising model
    for a given temperature and coupling
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
        Ac : np.array(D, d, D)
            MPS tensor of the MPS fixed point,
            with 3 legs ordered left-bottom-right,
            center gauge.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
    
        Returns
        -------
        M : float
            Expectation value of the magnetization at the given temperature
            and coupling.

    """
    return ncon((Fl, Ac, isingM(beta, J), np.conj(Ac), Fr), (
        [1, 3, 2], [2,7,5],[3,7,8,6],[1,6,4], [5,8,4]))


def isingZ(beta, J, Ac, Fl, Fr):
    """
    Computes the Ising model partition function for a given temperature and
    coupling
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
        Ac : np.array(D, d, D)
            MPS tensor of the MPS fixed point,
            with 3 legs ordered left-bottom-right,
            center gauge.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
    
        Returns
        -------
        Z : float
            Value of the partition function at the given temperature and
            coupling.

    """
    
    Z = ncon((Fl, Ac, isingO(beta, J), np.conj(Ac), Fr), (
        [1, 3, 2], [2,7,5],[3,7,8,6],[1,6,4], [5,8,4]))
    
    return Z


def isingExact(beta, J):
    """
    Exact Onsager solution for the 2d classical Ising Model

        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        magnetization : float
            Magnetization at given temperature and coupling.
        free : float
            Free energy at given temperature and coupling.
        energy : float
            Energy at given temperature and coupling.

    """
    theta = np.arange(0, np.pi/2, 1e-6)
    x = 2 * np.sinh(2 * J * beta) / np.cosh(2 * J * beta) ** 2
    if 1 - (np.sinh(2 * J * beta)) ** (-4) > 0:
        magnetization = (1 - (np.sinh(2 * J * beta)) ** (-4)) ** (1 / 8)
    else:
        magnetization = 0
    free = -1 / beta * (np.log(2 * np.cosh(2 * J * beta)) + 1 / np.pi * np.trapz(np.log(1 / 2 * (1 + np.sqrt(1 - x ** 2 * np.sin(theta) ** 2))), theta))
    K = np.trapz(1 / np.sqrt(1 - x ** 2 * np.sin(theta) ** 2), theta)
    energy = -J * np.cosh(2 * J * beta) / np.sinh(2 * J * beta) * (1 + 2 / np.pi * (2 * np.tanh(2 * J * beta) ** 2 - 1) * K)
    return magnetization, free, energy

In [ ]:
D = 12
d = 2
J = 1

print('Bond dimension: D =', D)
Al = createMPS(D, d)
# optimization parameters
tol = 1e-5

Ts = np.linspace(1., 3.4, 100)
magnetizations = []
magnetizationsExact = []
freeEnergies = []
freeEnergiesExact = []

for T in Ts:
    beta = 1/T
    O = isingO(beta, J)
    print('T={}'.format(T))
    
    lam, Al, Ac, Ar, C, Fl, Fr = vumpsMPO(O, D, A0=Al, tol=tol)
    magnetizations.append(np.abs(isingMagnetization(beta, J, Ac, Fl, Fr)/isingZ(beta, J, Ac, Fl, Fr)))
    magnetizationsExact.append(isingExact(beta, J)[0])
    freeEnergies.append(-np.log(lam)/beta)
    freeEnergiesExact.append(isingExact(beta, J)[1])


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5), dpi=120)
ax1.set_title('Magnetization as a function of the temperature')
ax1.set(xlabel=r'$T$', ylabel=r'$<M>$')
ax1.scatter(Ts, magnetizations, label = 'D={}'.format(D), c="hotpink", marker="^")
ax1.plot(Ts, magnetizationsExact, label = 'exact')

ax2.set_title('Free energy as a function of the temperature')
ax2.set(xlabel=r'$T$', ylabel=r'$f$')
ax2.scatter(Ts, freeEnergies, label = 'D={}'.format(D), c="hotpink", marker="^")
ax2.plot(Ts, freeEnergiesExact, label = 'exact')
plt.legend()
